In [ ]:
#bu kodda istediğim sqldeki tablo adını yazıp biraz kusurlu da olsa bir özet, max min değerler ve grafikleri direkt görüp 
#kafamda hangi grafik nasıl olmalı veya dağılımlar ne anlatıyor nasıl sunulsa daha iyi olur gibi sorulara cevap bulabilmem 
#acisindan zaman kazandiriyor. gelistirlmis görseller vs diger botta olacak 
import pyodbc
import pandas as pd
from sqlalchemy import create_engine
import matplotlib.pyplot as plt
import seaborn as sns

# SQL bağlantısı
server = 'MY_SERVER_NAME'
database = 'my_database'
conn_str = f"mssql+pyodbc://{server}/{database}?driver=ODBC+Driver+17+for+SQL+Server&trusted_connection=yes"
engine = create_engine(conn_str, fast_executemany=True)

def get_table_summary(table_name):
    """Tabloyu SQL'den çek, özetini yazdır ve uygun görseli oluştur."""
    try:
        # Tabloyu DataFrame'e çek
        df = pd.read_sql(f"SELECT * FROM dbo.{table_name}", engine)
        
        print(f"\n✅ Tablo başarıyla çekildi: {table_name}")
        print(f"Toplam satır sayısı: {len(df)}")
        print("\nKolon bilgileri:")
        print(df.dtypes)
        
        # Null değer analizi
        nulls = df.isnull().sum()
        null_pct = (nulls / len(df) * 100).round(2)
        
        print("\n📊 Null değer oranları (%):")
        print(null_pct[null_pct > 0])
        
        print("\n🔍 İlk 5 satır:")
        print(df.head())

        # ---------------------------
        # Akıllı Görselleştirme
        # ---------------------------
        plt.figure(figsize=(10, 6))

        if "event_date" in df.columns and "events_per_day" in df.columns:
            # Tarih bazlı → çubuk grafik
            df["event_date"] = pd.to_datetime(df["event_date"])
            sns.barplot(x="event_date", y="events_per_day", data=df, color="royalblue")
            plt.xticks(rotation=45)
            plt.title("Günlük Event Sayıları")
            
            max_val = df["events_per_day"].max()
            max_date = df.loc[df["events_per_day"].idxmax(), "event_date"]
            print(f"📝 Yorum: En yüksek etkinlik {max_date.date()} tarihinde gözlenmiş ({max_val} event).")

        elif "event" in df.columns and "event_count" in df.columns:
            # Kategori bazlı → bar chart
            sns.barplot(x="event", y="event_count", data=df, color="seagreen")
            plt.xticks(rotation=45)
            plt.title("Event Türlerinin Dağılımı")
            
            top_event = df.loc[df["event_count"].idxmax(), "event"]
            print(f"📝 Yorum: En çok gerçekleşen event '{top_event}'.")

        elif "event_hour" in df.columns and "events_per_hour" in df.columns:
            # Saat bazlı → bar chart
            sns.barplot(x="event_hour", y="events_per_hour", data=df, color="darkorange")
            plt.title("Saatlik Event Dağılımı")

            peak_hour = df.loc[df["events_per_hour"].idxmax(), "event_hour"]
            print(f"📝 Yorum: En yoğun saat {peak_hour}. saattir.")

        elif "month" in df.columns and "events_per_month" in df.columns:
            # Aylık → bar chart
            sns.barplot(x="month", y="events_per_month", data=df, color="purple")
            plt.title("Aylık Event Dağılımı")

            peak_month = df.loc[df["events_per_month"].idxmax(), "month"]
            print(f"📝 Yorum: En yoğun ay {peak_month}. ay olmuştur.")

        else:
            print("ℹ️ Bu tablo için özel bir görselleştirme tanımlanmadı.")

        plt.tight_layout()
        plt.show()
        
        return df
    
    except Exception as e:
        print(f"⚠️ Hata: {e}")

# ---------------------------
# BOT Döngüsü
# ---------------------------
while True:
    tablo = input("📂 Hangi tabloyu incelemek istersin? (çıkmak için 'q') >>> ")
    if tablo.lower() == 'q':
        print("Bot kapatılıyor... Görüşmek üzere 👋")
        break
    else:
        df = get_table_summary(tablo)

#girdigin tabloya gore otomatik gorsellestiriyor tabloyu getiriyor 
